<a href="https://colab.research.google.com/github/Pataweepr/applyML_vistec_2019/blob/master/hw2_homeDotTech_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Home dot tech 

ข้อมูลนี้เป็นข้อมูลเกี่ยวกับ ที่อยู่ในประเทศไทย 

พวกเราจะใช้ file จาก google drive ซึ่งต้องใช้ pyDrive


สามารถ อ่านการใช้งาน pydrive เบื้องต้นได้ตามนี้ 

[example pyDrive](https://github.com/Pataweepr/applyML_vistec_2019/blob/master/howToLoadDataWPyDrive.ipynb)

[link download file](https://drive.google.com/file/d/1X-cacRIF30acXyKotSmDZu5qc8Gs84Wm/view?usp=sharing)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
!unzip '/content/gdrive/My Drive/For_participants.zip'
!ls

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.stats import mode
from sklearn import preprocessing

we use "delimiter= ';' " to split colums

In [0]:
def readDataFromDrive(file_name):
  raw_data = pd.read_csv(file_name,delimiter= ';')
  return raw_data;

files that we use in this exercise

* userLog  :  มันการเป็นเก็บ log ของ user ทั้งหมดที่เข้ามาในระบบ ว่า user เข้าผ่านอุปกรณื อะไรบ้างแล้วเข้าไปดูโปรเจคไหน
* project_main : เป็นการบอกรายละเอียดแต่ละ project ว่ามีข้อมูลอะไรบ้าง เกี่ยวกับตำแหน่ง หรือ ตัวของ ราคา
* project_facility : บอก facility ของแต่ละโปรคเจคว่าเป็นอย่างไร





# TODO#1: เปลี่ยนข้อมูลเป็นตัวเลข

ข้อมูบบที่เราจะพิจารณาต่อไปนี้เป็นข้อมูล

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
project_main = readDataFromDrive('project_main.csv')

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
print('project_main')
project_main = readDataFromDrive('project_main.csv')
project_main = project_main.loc[project_main['province_id'] == 10]
print(project_main.shape)
project_main = project_main.drop(columns=['project_land_size_ngan', 'project_land_size_wa'])
project_main.loc[project_main["project_status"] == "A", "project_status"] = 1
project_main.loc[project_main["project_status"] == "U", "project_status"] = 0
project_main = project_main.convert_objects(convert_numeric=True)
project_main.head()
        </code>
      </pre>
</details>



# TODO#2: แก้ไขข้อมูล data frame ของ projet_main ให้ไม่มี NAN

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
project_main = project_main.dropna(subset=['lat'])
project_main["starting_price"] = project_main["starting_price"].fillna(project_main["starting_price"].mode().iloc[0])
        </code>
      </pre>
</details>

## TODO#3: load ข้อมูล facility
ลอง print ข้อมูลดูเราจะเห็นว่าข้อมูล และอธิบายว่าเหมาะสมกับการใช้หรือไม่


In [0]:

project_facility = readDataFromDrive('project_facility.csv')
project_facility.head()


## TODO#4: แปลงข้อมูล fac ให้เหมาะแก้การใช้งาน 
เนื่องจากข้อมูลไม่ได้เป็นรูปแบบของตารางที่ใช้ได้ง่าย [pd.crosstab](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.crosstab.html) เพราะ เราต้องการตารางที่บอกถึง facility ว่ามีหรือไม่ในทุกแบบของ facility

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
project_facility_table 

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
project_facility_table = pd.crosstab(project_facility['project_id'], project_facility['facility_id'], rownames=['project_id'], colnames=['facility_id'])
print(project_facility_table.index)
project_facility_table.head()
        </code>
      </pre>
</details>

## TODO#5: กำจัดข้อมูลที่ใช้ไม่ได้ ออกไป เลือกใช้แต่ข้อมูลที่ครบ

เราจำเป็นต้องแยกข้อมูลให้เหลือ project ที่มีข้อมูลครบ เท่านั้น  เราสามารถใช้ [pandas.DataFrame.isin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) เพื่อช่วยในการหา วา่มีข้อมูลร่วมกันได้ในการแยกข้แมูลออกมา

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
project_facility_table = project_facility_table.loc[project_facility_table.index.isin(project_main['project_id'])]
project_main = project_main.loc[project_main['project_id'].isin(project_facility_table.index.astype(int))]
project_main = project_main.sort_values('project_id')
print(project_facility_table.shape)
print(project_main.shape)
project_facility_table.head()
        </code>
      </pre>
</details>

In [0]:
project_main.head()

## TODO#6: ลองดูข้อมูล จาก user log 



ลอง check ข้อมูลของ log data 

In [0]:
# care values province_id
# we use only bangkok  province_id = 10 
### load data ###############################################################################
print('userLog')
userLog = readDataFromDrive('userLog_201801_201802_for_participants.csv') 
userLog.head()


In [0]:
print(userLog.shape)

## clean project in  userLog

แก้ไขให้เลอืก แต่ project เรามีข้อมูลครบ

In [0]:

################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################



<details>
    <summary>SOLUTION HERE!</summary>
    <code>
  userLog = userLog.loc[userLog['project_id'].isin(project_facility_table.index.astype(int))]
  </code>
</details>



## TODO#7: ข้อมูล histrogram ของตัว user และ project ที่ user ดู

จง check ดูความบ่อยของการใช้ user และ plot histogram ออกมา 
เราสามารถใช้ [pd.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) เพื่อช่วยการนับจำนวนได้

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
userLog_freq_user

<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
  <code>
  userLog_freq_user = userLog['userCode'].value_counts()
  userLog_freq_user_np  = np.array(userLog_freq_user.values)
  print(userLog_freq_user_np.shape)
  userLog_freq_user.head()
  </code>
  </pre>
</details>




จงแสดง histogram ของความถี่ของ user และ ใช้ pd.head() เพื่อดูค่าช่วงต้นๆ ของ histogram

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

## TODO#8: จงหาค่า medium และ mode ของ ความถี่ในการเข้าใช้ ของ user
จากค่าที่พิจารณา  จงหา mode medium percentile ที่ 95 [np.percentile](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.percentile.html)

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


## TODO#9: 

จากกราฟ histogram ที่ได้ละ ค่า mode medium และ ค่าสูงสุดที่ user ใช้ จงบอกปัญหาของข้อมูล ของ userlog ว่าเกิดอะไรขึ้น

เขียนคำตอบ :

## TODO#10: 

จะสังเกตว่าค่า ของ histogram user นั้นมี user ประหลาดอยู่ดังนั้น เราจะกำลังข้อมูลที่ไม่ดีออกไป โดยการตัด เปอร์เซ็นไทน์ 90 ของการใช้ของ user

ใช้ [pd.index](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Index.html) ในการดึงค่า index ของข้อมูลออกมา เพื่อช่วยในการจัดการข้อมูลได้ 

ต้องตรวจสอบ ด้วย histogram ด้วย

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
  <code>

userLog_freq_user = userLog_freq_user.loc[userLog_freq_user <= 40 ]
userLog_freq_user = userLog_freq_user.loc[userLog_freq_user >= 5 ]
userLog_freq_user_np = np.array(userLog_freq_user.values)

n, bins, patches = plt.hist(userLog_freq_user_np, 100, density=True, facecolor='g', alpha=0.75)
plt.show()

userLog = userLog.loc[userLog['userCode'].isin(userLog_freq_user.index)]
  </code>
  </pre>
</details>



#optional 

พวกเราสามารถ filter ข้อมูลด้วย จำนวนการ เข้าดูในแต่ละวันได้หรือไม่



In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

------

## TODO#11:

เราจะสร้าง recommendation system โดยจะแบ่งข้อมูลจาก โดยให้ข้อมูลในสัปดาห์สุดท้ายนั้น เป็นตรวของ test set ที่เหลือนั้นเป็น train set

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
userLog_freq_day = pd.crosstab(userLog['userCode'], [userLog['year'], userLog['month'], userLog['day']], rownames =['userCode'], colnames=['year','month','day'])
userLog_freq_day.head()

<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
userLog_freq_day = pd.crosstab(userLog['userCode'], [userLog['year'], userLog['month'], userLog['day']], rownames =['userCode'], colnames=['year','month','day'])

userLog_freq_day.head()

</code>
</pre>
</details>







# split train and test set

พวก เราจะใช้ user ที่มาใช้ในเดือนสุดท้ายเท่านั้น
train set ใช้วันที่ 15- 18
test set ใช้วันที่ 19 -22 

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>

userLog_last_month  = userLog.loc[userLog['month'] == 2]
userLog_train = userLog_last_month.loc[ userLog_last_month['day'] >=15]
userLog_train = userLog_train.loc[ userLog_train['day'] < 19]
userLog_test = userLog_last_month.loc[ userLog_last_month['day'] >= 19]

print(userLog_last_month.shape)
print(userLog.shape)

</code>
</pre>
</details>




#  สร้าง dataframe ของข้อมูลที่สื่อว่า user แต่ละคน ดู project ไหนบ้าง จาก train set และ test set
## hint : use pd.crosstab

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
freq_user_seen_proj_train

<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
freq_user_seen_proj_train = pd.crosstab(userLog_train['userCode'], userLog_train['project_id'], rownames=['userCode'], colnames=['project_id'])
freq_user_seen_proj_train_np = np.array(freq_user_seen_proj_train.values)
#print(freq_user_seen_proj_train_np)
print(freq_user_seen_proj_train_np.shape)

freq_user_seen_proj_train.head()
</code>
</pre>
</details>



In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
freq_user_seen_proj_test

<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
freq_user_seen_proj_test = pd.crosstab(userLog_test['userCode'], userLog_test['project_id'], rownames=['userCode'], colnames=['project_id'])
freq_user_seen_proj_test_np = np.array(freq_user_seen_proj_test.values)
#print(freq_user_seen_proj_test_np)
print(freq_user_seen_proj_test_np.shape)

freq_user_seen_proj_test.head()
</code>
</pre>
</details>




## TODO#11:  function to normalize data
[Preprocessing data](https://scikit-learn.org/stable/modules/preprocessing.html?fbclid=IwAR3jCj2F4T1qZqgqwM-rYSlLC_WymKPa4tF4DYk5AfcwlIL0cyiIdi4VMpA#preprocessing-scaler)


In [0]:
def normalize(x):
  ################################## #############################################
  #                            WRITE YOUR CODE BELOW                             #
  ################################################################################
  return

<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
def normalize(x):
  min_max_scaler = preprocessing.MinMaxScaler()
  x_nor = min_max_scaler.fit_transform(x)
  return x_nor;
</code>
</pre>
</details>



# recommendation system
พวกเราจะสร้าง recomentation sys โดยเราจะใช้ NNในการสร้างขึ้นมา

[Nearest Neighbors example](https://scikit-learn.org/stable/modules/neighbors.html)




## TODO#11:  function ที่สร้าง average feature จาก project ที่ user แต่ละคนดู

[Preprocessing data](https://scikit-learn.org/stable/modules/preprocessing.html?fbclid=IwAR3jCj2F4T1qZqgqwM-rYSlLC_WymKPa4tF4DYk5AfcwlIL0cyiIdi4VMpA#preprocessing-scaler)

In [0]:
# recommentdation system
def feature_sel(userLog_data_frame,user_code):
  ################################## #############################################
  #                            WRITE YOUR CODE BELOW                             #
  ################################################################################
  return


<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
def feature_sel(userLog_data_frame,user_code):
  np_data = np.zeros((user_code.shape[0], 3 + project_facility_table.shape[1]))
  
  for i in np.arange(user_code.shape[0]):
    user_sel = user_code[i]
    #print(user_sel)
    userLog_dataF_sel = userLog_data_frame.loc[userLog_data_frame['userCode'] == user_sel]
    proj_id_sel = userLog_dataF_sel['project_id'].values
    input_np_data = np.zeros((proj_id_sel.shape[0],3 + project_facility_table.shape[1]))
    # print(proj_id_sel)
    for j in np.arange(proj_id_sel.shape[0]):
      user_proj_far = project_facility_table.loc[project_facility_table.index == proj_id_sel[j]]
      user_proj_main = project_main.loc[project_main['project_id'] == proj_id_sel[j]]
      input_np_data[j,:] = np.hstack(( np.array(user_proj_main[["lat","lon","starting_price"]].values) ,  np.array(user_proj_far.values) ))
  np_data[i,:] = np.mean(input_np_data,axis=0)
  return np_data;
</code>
</pre>
</details>




# TODO#11: 

run example จาก feature ที่เรากำหนด ขึ้นจากหัวข้อด้านบน



In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
project_main_np_data_nor = feature_sel(userLog_train,freq_user_seen_proj_train.index)
project_main_np_data_nor = normalize(project_main_np_data_nor)
print(project_main_np_data_nor.shape)

nbrs_proj = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(project_main_np_data_nor)
# print(project_main_np_data_nor)

distances, indices = nbrs_proj.kneighbors(project_main_np_data_nor)
</code>
</pre>
</details>



ลองสรา้ง  recommendation system จาก การดูของ user ว่า ใกล้เคียง กับ user คนไหนบ้าง และ recommend ตาม proj ที่ได้ดู



In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################


<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
  <code>
  freq_user_seen_proj_train_nor = normalize(freq_user_seen_proj_train_np) 
  nbrs_user_numberical = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(freq_user_seen_proj_train_nor) 
  distances, indices = nbrs_user_numberical.kneighbors(freq_user_seen_proj_train_nor)
  </code>
  </pre>
</details>




เปลี่ยนเป็นข้อมูลว่าแต่ละคนดูอะไรบ้าง และใช้ jaccard เป็น distance (ข้อมูลในการวัดแบบ jaccard ต้องเป็น boolean) [distance function](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric)

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################



<details>
    <summary>SOLUTION HERE!</summary>
  <pre>
<code>
nbrs_user = NearestNeighbors(n_neighbors= 6, algorithm='auto',metric='jaccard' ).fit(freq_user_seen_proj_train_np >= 1)
distances, indices = nbrs_user.kneighbors(freq_user_seen_proj_train_np >= 1)
</code>
</pre>
</details>




แสดงผลการ recommendation โดยดูจากผลลัพธ์ว่าใกล้เคียงอย่างไรบ้าง

In [0]:
################################## #############################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################

# evaluation 

ในการ recommendation จริงๆนั้นมีการ วัดผลการ recommend โดยใช้รูปแบบนี้เป็นหลักลองแสดงการวัดผลโดยใช้ matrix นี่ดู [Mean Average Precision](http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html?fbclid=IwAR2UOfz0a_2Ig60aQ2HszgZq63nch96Hbzod2q54kDZRZT_mPzdsxJXyPj0)

การวัดผล recommendation system เราจะดูจาก project ที่เขาได้ ดูตัวใหม่เท่านั้น

เราจะใช้ function ตัวล่างเพื่อตรวจสอบการวัดผล

1.   proj_seen : ตัวสร้าง dict ของ project ที่ user ดู
    *   data_freq_seen (input) : df ของความถี่ที่ user ดูแต่ละ project ex.freq_user_seen_proj_train, freq_user_seen_proj_test
    *   output : dict ของ project ที่ user ดู
2.   index_to_usercode : ตัวสร้าง dictonary ของ  Nearest Neighbors
    *   nn_index (input) : ผลลัพธ์จากการ run Nearest Neighbors (ค่า indices ที่เป็น output ของ NearestNeighbors.kneighbors)
    *   user_code_list (input) : ค่า freq_user_seen_proj_train.index
    *   output :  dict ของ  Nearest Neighbors ว่า user ไหนใกล้กับคนไหนเป็นอันดับ
3.   proj_recommend : ตัวสร้างว่าเราควร recommend ตัวไหน
    *   nn_dict (input) : output ของ index_to_usercode
    *   dict_seen_train (input) : output ของ proj_seen จาก freq_user_seen_proj_train (train set)
    *   output : dict ของ การ recommend แต่ละ user (ยังไม่ตัดตัวซ้ำ)
4.   proj_repeat : ตัดตัวซ้ำ (project ที่ userเคยดูไปแล้ว)
    *   dict_train (input) : dict การดูของuser ที่ช้ train model 
    *   dict_test (input) : dict การดูของ user ที่ใช้ test หรือ recommend (มีตัวซ้ำกับ dict_train)
    *   output : dict_test ที่ตัดค่าซ้ำใน dict_train
5.   mean_average_precision : ตัวหาค่า map at k 
    *   dict_recomment (input) : dict ที่ต้องการ recommend (ที่ตัดตัวซ้ำออก)
    *   dict_test (input) : dict นำมา test การ recommend (ที่ตัดตัวซ้ำออก)
    *   k (input) : จำนวนมากสุด ที่ จะ recommend
    *   output : ค่า map at K


In [0]:
def proj_recommend(nn_dict,dict_seen_train):
  nn_proj = {}
  for user_code_nn in nn_dict:
    list_proj = []
    for user_in_list in nn_dict[user_code_nn]:
      list_proj = list_proj + list(dict_seen_train[user_in_list])
    nn_proj[user_code_nn] = list_proj
  return nn_proj

####################################################################

def index_to_usercode(nn_index,user_code_list):
  nn_dict = {}
  for i in np.arange(nn_index.shape[0]):
    nn_list = []
    nn_name = user_code_list[i]
    for j in np.arange(nn_index.shape[1]):
      nn_list.append(user_code_list[j])
    nn_dict[nn_name] = nn_list
  return nn_dict

####################################################################

def proj_repeat(dict_train,dict_test):
  dict_out = {} 
  for user_code in dict_test:
    if user_code in dict_train:
      list_proj_train = np.array(dict_train[user_code])
      list_proj_test = np.array(dict_test[user_code])
      bool_list = np.isin(list_proj_test ,list_proj_train)
      dict_out[user_code] = list(list_proj_test[bool_list])
  return dict_out

####################################################################

def proj_seen(data_freq_seen):
  all_colums = data_freq_seen.columns
  all_index = data_freq_seen.index
  output_list = {}
  for index in np.arange(len(all_index)):
    np_array = np.array(data_freq_seen.iloc[index])
    output_list[all_index[index]] = all_colums[np_array > 0].values
  return output_list

####################################################################

def mean_average_precision(dict_recomment,dict_test,k):
  list_map_at_k = []
  for user_code in dict_test:
    list_map_at_k_user_i = []
    list_proj_rec = np.array(dict_recomment[user_code])
    list_proj_test = np.array(dict_test[user_code])
    
    ##########
    
    for i in np.arange(k):
      if(i == len(list_proj_rec) or len(list_proj_test) == 0 or len(list_proj_rec) == 0):
        break
      bool_list = np.isin(list_proj_rec[0:i+1],list_proj_test)
      list_map_at_k_user_i.append(np.sum(bool_list)/(i+1))
    
    ##########

    number = np.mean(np.array(list_map_at_k_user_i))
    
    if(~np.isnan(number)):
      list_map_at_k.append(number)

  return np.mean(np.array(list_map_at_k))

# ตรวจดูคำตอบ NN

# example evaluation

In [0]:
dist_user_test = proj_seen(freq_user_seen_proj_test)
dist_user_train = proj_seen(freq_user_seen_proj_train)
dist_user_test = proj_repeat(dist_user_train,dist_user_test)

nn_dict_rec = index_to_usercode(indices,freq_user_seen_proj_train.index)

dist_user_rec = proj_recommend(nn_dict_rec,dist_user_train)
dist_user_rec = proj_repeat(dist_user_train,dist_user_rec)

#################################################################

k = 3
map_k = mean_average_precision(dist_user_rec,dist_user_test,k)
print('Your result : ', map_k)

------
